In [ ]:
! pip install -U kaleido && pip install git+https://github.com/jkminder/pycolors.git && pip install -e .../dictionary_learning

In [14]:
%load_ext autoreload
%autoreload 2
import sys

sys.path.append("../")
from transformers import AutoModelForCausalLM, AutoTokenizer
from dictionary_learning.cache import ActivationCache
from dictionary_learning.dictionary import CrossCoder
from datasets import load_from_disk, load_dataset
from loguru import logger
import torch as th
from nnsight import LanguageModel
from pathlib import Path
from pycolors import TailwindColorPalette
import os
# from tools.latent_analysis import plot_feature_freq, plot_feature_diff
from pathlib import Path
from tools.utils import load_latent_df, push_latent_df
# from dlabutils import model_path

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
crosscoder = "connor"

df = load_latent_df(crosscoder)
df

feature_df.csv:   0%|          | 0.00/4.78M [00:00<?, ?B/s]

,Unnamed: 0,dec_base_norm,dec_instruct_norm,dec_norm_diff,tag,enc_base_norm,enc_instruct_norm,enc_norm_diff,dec_cos_sim,enc_cos_sim,...,fw_max_act,freq,fw_freq,lmsys_freq,fw_dead,lmsys_dead,dead,lmsys_max_act_unscaled,fw_max_act_unscaled,max_act_unscaled
0,0,0.929085,0.892494,0.519692,Shared,2.030810,1.969423,0.515114,0.960600,0.063047,...,83.447430,0.003266,0.002839,0.003797,False,False,False,45.450397,45.810493,45.810493
1,1,0.784714,0.798819,0.491171,Shared,1.524475,1.531144,0.497822,0.978069,0.269552,...,146.685440,0.002300,0.003083,0.001329,False,False,False,78.601830,92.631720,92.631720
2,2,0.883436,0.865628,0.510079,Shared,2.110544,1.985030,0.529735,0.988032,0.313676,...,110.751840,0.001533,0.001782,0.001225,False,False,False,72.058914,63.320652,72.058914
3,3,0.821927,0.791367,0.518590,Shared,1.550208,1.518734,0.510152,0.957581,0.477391,...,86.806190,0.001080,0.001303,0.000803,False,False,False,57.714005,53.806805,57.714005
4,4,1.121779,0.590787,0.736674,Other,1.909612,1.680501,0.559989,0.908485,-0.145423,...,52.893753,0.002046,0.000299,0.004212,False,False,False,43.521744,30.885653,43.521744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16379,16379,0.956823,0.833378,0.564508,Shared,1.627625,1.593929,0.510351,0.916842,0.045734,...,51.962177,0.001362,0.001722,0.000916,False,False,False,32.106194,29.025879,32.106194
16380,16380,0.882795,1.138962,0.387544,Other,1.814163,1.759773,0.514990,0.910427,-0.214700,...,48.425034,0.008391,0.006038,0.011309,False,False,False,21.762463,23.951962,23.951962
16381,16381,0.849927,0.815499,0.520253,Shared,1.372850,1.329197,0.515899,0.980973,0.563518,...,121.079300,0.001164,0.001277,0.001025,False,False,False,68.441734,72.701744,72.701744
16382,16382,0.804095,0.803279,0.500507,Shared,1.615137,1.574377,0.512618,0.974792,0.244192,...,126.206320,0.002442,0.003175,0.001534,False,False,False,81.893480,78.517090,81.893480


In [16]:
df.columns

Index(['Unnamed: 0', 'dec_base_norm', 'dec_instruct_norm', 'dec_norm_diff',
       'tag', 'enc_base_norm', 'enc_instruct_norm', 'enc_norm_diff',
       'dec_cos_sim', 'enc_cos_sim', 'twin_latent',
       'cosim_component_sizes_base', 'cosim_component_sizes_chat', 'max_act',
       'lmsys_max_act', 'fw_max_act'],
      dtype='object')

In [17]:
stats_path = Path("/workspace/data/results/latent-stats/ckkissane_crosscoder-gemma-2-2b-model-diff")
stats_fineweb = th.load(stats_path / "fineweb.pt", map_location="cpu")
stats_lmsys = th.load(stats_path / "lmsys.pt", map_location="cpu")

/tmp/ipykernel_1751/691379874.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stats_fineweb = th.load(stats_path / "fineweb.pt", map_location="cpu")
/tmp/ipykernel_1751/

In [1]:
stats_fineweb.rescaled.joint

NameError: name 'stats_fineweb' is not defined

In [19]:
df.columns

Index(['Unnamed: 0', 'dec_base_norm', 'dec_instruct_norm', 'dec_norm_diff',
       'tag', 'enc_base_norm', 'enc_instruct_norm', 'enc_norm_diff',
       'dec_cos_sim', 'enc_cos_sim', 'twin_latent',
       'cosim_component_sizes_base', 'cosim_component_sizes_chat', 'max_act',
       'lmsys_max_act', 'fw_max_act'],
      dtype='object')

In [20]:
import numpy as np
df["freq"] = (stats_fineweb.rescaled.joint.non_zero_counts + stats_lmsys.rescaled.joint.non_zero_counts) / (stats_fineweb.rescaled.joint.total_tokens + stats_lmsys.rescaled.joint.total_tokens)
df["fw_freq"] = stats_fineweb.rescaled.joint.non_zero_counts / stats_fineweb.rescaled.joint.total_tokens
df["lmsys_freq"] = stats_lmsys.rescaled.joint.non_zero_counts / stats_lmsys.rescaled.joint.total_tokens
df["fw_dead"] = stats_fineweb.rescaled.joint.non_zero_counts < 10
df["lmsys_dead"] = stats_lmsys.rescaled.joint.non_zero_counts < 10
df["dead"] = df["fw_dead"] | df["lmsys_dead"]
df["lmsys_max_act"] = stats_lmsys.rescaled.joint.max_activation
df["fw_max_act"] = stats_fineweb.rescaled.joint.max_activation
df["max_act"] = np.maximum(df["lmsys_max_act"], df["fw_max_act"])
df["lmsys_max_act_unscaled"] = stats_lmsys.normal.joint.max_activation
df["fw_max_act_unscaled"] = stats_fineweb.normal.joint.max_activation
df["max_act_unscaled"] = np.maximum(df["lmsys_max_act_unscaled"], df["fw_max_act_unscaled"])
df

,Unnamed: 0,dec_base_norm,dec_instruct_norm,dec_norm_diff,tag,enc_base_norm,enc_instruct_norm,enc_norm_diff,dec_cos_sim,enc_cos_sim,...,fw_max_act,freq,fw_freq,lmsys_freq,fw_dead,lmsys_dead,dead,lmsys_max_act_unscaled,fw_max_act_unscaled,max_act_unscaled
0,0,0.929085,0.892494,0.519692,Shared,2.030810,1.969423,0.515114,0.960600,0.063047,...,83.447433,0.003266,0.002839,0.003797,False,False,False,45.450397,45.810493,45.810493
1,1,0.784714,0.798819,0.491171,Shared,1.524475,1.531144,0.497822,0.978069,0.269552,...,146.685440,0.002300,0.003083,0.001329,False,False,False,78.601830,92.631721,92.631721
2,2,0.883436,0.865628,0.510079,Shared,2.110544,1.985030,0.529735,0.988032,0.313676,...,110.751839,0.001533,0.001782,0.001225,False,False,False,72.058914,63.320652,72.058914
3,3,0.821927,0.791367,0.518590,Shared,1.550208,1.518734,0.510152,0.957581,0.477391,...,86.806190,0.001080,0.001303,0.000803,False,False,False,57.714005,53.806805,57.714005
4,4,1.121779,0.590787,0.736674,Other,1.909612,1.680501,0.559989,0.908485,-0.145423,...,52.893753,0.002046,0.000299,0.004212,False,False,False,43.521744,30.885653,43.521744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16379,16379,0.956823,0.833378,0.564508,Shared,1.627625,1.593929,0.510351,0.916842,0.045734,...,51.962177,0.001362,0.001722,0.000916,False,False,False,32.106194,29.025879,32.106194
16380,16380,0.882795,1.138962,0.387544,Other,1.814163,1.759773,0.514990,0.910427,-0.214700,...,48.425034,0.008391,0.006038,0.011309,False,False,False,21.762463,23.951962,23.951962
16381,16381,0.849927,0.815499,0.520253,Shared,1.372850,1.329197,0.515899,0.980973,0.563518,...,121.079300,0.001164,0.001277,0.001025,False,False,False,68.441734,72.701744,72.701744
16382,16382,0.804095,0.803279,0.500507,Shared,1.615137,1.574377,0.512618,0.974792,0.244192,...,126.206322,0.002442,0.003175,0.001534,False,False,False,81.893478,78.517090,81.893478


In [21]:
df["freq"].min()

np.float32(2.9870382e-05)

In [36]:
df["dead"].sum()

np.int64(0)

In [24]:
df["max_act"].dtype

dtype('float32')

In [27]:
push_latent_df(df, crosscoder)

/workspace/julian/repositories/representation-structure-comparison/notebooks/../tools/utils.py:389: UserWarning: Column lmsys_max_act has different dtype in original and new df
  warnings.warn(
/workspace/julian/repositories/representation-structure-comparison/notebooks/../tools/utils.py:389: UserWarning: Column fw_max_act has different dtype in original and new df
  warnings.warn(
/workspace/julian/repositories/representation-structure-comparison/notebooks/../tools/utils.py:389: UserWarning: Column max_act has different dtype in original and new df
  warnings.warn(


Added columns in uploaded df: {'fw_max_act_unscaled', 'max_act_unscaled', 'lmsys_dead', 'lmsys_max_act_unscaled', 'fw_dead', 'fw_freq', 'freq', 'dead', 'lmsys_freq'}
Column lmsys_max_act has different values in original and new df
Column fw_max_act has different values in original and new df
Column max_act has different values in original and new df
Commit message: None


In [41]:
os.environ["HF_HOME"]

'/workspace/julian/hf_cache'

In [8]:
cc_name = "l13-mu4.1e-02-lr1e-04"
stats_path = Path("../stats/")
save_dir = Path(f"../plots/") / model_name
base_dir = Path(f"../results/eval_crosscoder/{cc_name}_ae_final/data")

os.makedirs(save_dir, exist_ok=True)

stats_fineweb = th.load(stats_path / model_name / "fineweb.pt", map_location="cpu")
stats_lmsys = th.load(stats_path / model_name / "lmsys.pt", map_location="cpu")

In [ ]:
for rescaled in [True, False]:
    for split in ["joint", "instruction", "base"]:
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name="base",
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name="instruction",
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name="shared",
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )
        plot_feature_freq(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name=None,
            split=split,
            rescaled=True,
            topk=200,
            indices_path=base_dir,
            save=True,
        )

In [ ]:
for rescaled in [True, False]:
    for group in ["base", "instruction", "shared", None]:
        plot_feature_diff(
            save_dir,
            stats_fineweb,
            stats_lmsys,
            group_name=group,
            rescaled=rescaled,
            indices_path=base_dir,
            save=True,
        )

In [11]:
stats_lmsys

CombinedFeatureStatistics(rescaled=FeatureStatistics(base=FeatureStatistic(avg_activation=tensor([2.2304e-03, 1.0332e-03, 7.5188e-06,  ..., 9.1480e-04, 4.0949e-04,
        1.1720e-04]), non_zero_counts=tensor([10947,  6632,   107,  ...,  1240, 12559,   622]), total_tokens=tensor(11930591), is_normalized=True), instruction=FeatureStatistic(avg_activation=tensor([0.0147, 0.0014, 0.0002,  ..., 0.0016, 0.0015, 0.0003]), non_zero_counts=tensor([133964,   6756,   2800,  ...,   2771,   9807,   1869]), total_tokens=tensor(11930591), is_normalized=True), joint=FeatureStatistic(avg_activation=tensor([2.0735e-03, 2.1034e-03, 1.0883e-05,  ..., 4.4947e-03, 2.0572e-03,
        4.9375e-04]), non_zero_counts=tensor([8070, 1591,   46,  ..., 1170, 3028,  374]), total_tokens=tensor(11930591), is_normalized=True), abs_activation_diff=tensor([-0.0125, -0.0004, -0.0002,  ..., -0.0007, -0.0011, -0.0001]), rel_activation_diff=tensor([0.0717, 0.4868, 0.0349,  ..., 0.2005, 0.5493, 0.2123]), either_non_zero_coun

In [25]:
# Get Fossil Features
# Load instruction only indices
from tools.feature_utils import indices_to_mask, mask_to_indices

indices = th.load(base_dir / "only_it_decoder_feature_indices.pt", map_location="cpu")
mask = indices_to_mask(indices, stats_lmsys.rescaled.base.avg_activation.shape[0])

/tmp/ipykernel_1912165/3879260724.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  indices = th.load(base_dir / "only_it_decoder_feature_indices.pt", map_location="cpu")


In [26]:
lmsys_it_fossil_mask = mask & (stats_lmsys.rescaled.rel_activation_diff > 0.95)
lmsys_it_fossil_indices = mask_to_indices(lmsys_it_fossil_mask)
len(lmsys_it_fossil_indices)

45

In [27]:
th.save(lmsys_it_fossil_indices, base_dir / "lmsys_it_fossil_indices.pt")

In [28]:
base_dir / "lmsys_it_fossil_indices.pt"

PosixPath('/dlabscratch1/cdumas/representation-structure-comparison/notebooks/results/eval_crosscoder/l13-mu4.1e-02-lr1e-04_ae_final/data/lmsys_it_fossil_indices.pt')

In [31]:
for fossil in lmsys_it_fossil_indices:
    assert fossil in indices

In [ ]:
stats_lmsys

In [ ]:
model_name = "l13-mu4.1e-02-lr1e-04"
stats_path = Path(
    f"/dlabscratch1/jminder/repositories/representation-structure-comparison/stats/"
)
save_dir = (
    Path(
        f"/dlabscratch1/jminder/repositories/representation-structure-comparison/plots/"
    )
    / model_name
)
base_dir = Path(
    f"/dlabscratch1/cdumas/representation-structure-comparison/notebooks/results/eval_crosscoder/l13-mu4.1e-02-lr1e-04_ae_final/data"
)

os.makedirs(save_dir, exist_ok=True)

stats_fineweb = th.load(stats_path / model_name / "fineweb.pt").to("cpu")
stats_lmsys = th.load(stats_path / model_name / "lmsys.pt").to("cpu")

In [ ]:
lmsys_it_fossil_mask = mask & (stats_lmsys.rescaled.rel_activation_diff > 0.95)
lmsys_it_fossil_indices = mask_to_indices(lmsys_it_fossil_mask)
len(lmsys_it_fossil_indices)

In [27]:
th.save(lmsys_it_fossil_indices, base_dir / "lmsys_it_fossil_indices.pt")

In [ ]:
base_dir / "lmsys_it_fossil_indices.pt"

In [31]:
for fossil in lmsys_it_fossil_indices:
    assert fossil in indices

In [ ]:
stats_lmsys

In [ ]:
# Get Fossil Features
# Load instruction only indices
from tools.feature_utils import indices_to_mask, mask_to_indices

indices = th.load(base_dir / "only_it_decoder_feature_indices.pt", map_location="cpu")
mask = indices_to_mask(indices, stats_lmsys.rescaled.base.avg_activation.shape[0])

In [ ]:
lmsys_it_fossil_mask = mask & (stats_lmsys.rescaled.rel_activation_diff > 0.95)
lmsys_it_fossil_indices = mask_to_indices(lmsys_it_fossil_mask)
len(lmsys_it_fossil_indices)

In [27]:
th.save(lmsys_it_fossil_indices, base_dir / "lmsys_it_fossil_indices.pt")

In [ ]:
base_dir / "lmsys_it_fossil_indices.pt"

In [31]:
for fossil in lmsys_it_fossil_indices:
    assert fossil in indices